# One shot learning with Siamese Network

In [1]:
!pip install tensorflow-gpu

In [2]:
!pip install tf-nightly 

## Import os, numpy for loading images

In [3]:
import sys
import numpy as np
import pandas as pd
import pickle
import os
import time
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
from sklearn.utils import shuffle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['shuffle', 'concatenate']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
!pip install awscli

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!unzip -uq "/content/drive/My Drive/siamese/images_background.zip" -d "/content/drive/My Drive/image_background"

## Load images into tensors

In [9]:
x = []
y = []
cat_dict = {}
lang_dict = {}
n=0
for a in os.listdir('/content/drive/My Drive/image_background/images_background'):
    print("Loading alphabet" , a)
    lang_dict[a] = [n , None]
    a_path = os.path.join('/content/drive/My Drive/image_background/images_background',a)
    
    for c in os.listdir(a_path):
        cat_dict[n] = (a,c)
        c_images = []
        c_path = os.path.join(a_path,c)
        
        for c_20 in os.listdir(c_path):
            c_20_path = os.path.join(c_path , c_20)
            img = imread(c_20_path)
            c_images.append(img)
            y.append(n)
            
        x.append(np.stack(c_images))
        lang_dict[a][1] = n
        n = n + 1       

Loading alphabet Bengali
Loading alphabet Blackfoot_(Canadian_Aboriginal_Syllabics)
Loading alphabet Braille
Loading alphabet Burmese_(Myanmar)
Loading alphabet Cyrillic
Loading alphabet Early_Aramaic
Loading alphabet Futurama
Loading alphabet Grantha
Loading alphabet Greek
Loading alphabet Gujarati
Loading alphabet Hebrew
Loading alphabet Inuktitut_(Canadian_Aboriginal_Syllabics)
Loading alphabet Japanese_(hiragana)
Loading alphabet Japanese_(katakana)
Loading alphabet Korean
Loading alphabet Latin
Loading alphabet Malay_(Jawi_-_Arabic)
Loading alphabet Mkhedruli_(Georgian)
Loading alphabet N_Ko
Loading alphabet Ojibwe_(Canadian_Aboriginal_Syllabics)
Loading alphabet Sanskrit
Loading alphabet Syriac_(Estrangelo)
Loading alphabet Tagalog
Loading alphabet Tifinagh
Loading alphabet Alphabet_of_the_Magi
Loading alphabet Anglo-Saxon_Futhorc
Loading alphabet Arcadian
Loading alphabet Armenian
Loading alphabet Asomtavruli_(Georgian)
Loading alphabet Balinese


In [10]:
y = np.vstack(y)

In [11]:
x = np.stack(x)

In [12]:
x.shape

(964, 20, 105, 105)

In [13]:
y.shape

(19280, 1)

In [14]:
print(y)

[[  0]
 [  0]
 [  0]
 ...
 [963]
 [963]
 [963]]


## Analysis of the shapes

In [15]:
y[0:20] , y[20:40]

(array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]), array([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]]))

In [16]:
x[0][1]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

## Create Batches

In [17]:
def generate_mini_batch(batchsize , x):
    n_chars , n_examples, width, height = x.shape
    alphabet = random.choice(n_chars,size=(batchsize,),replace=False)
    pairs=[np.zeros((batchsize, height, width,1)) for i in range(2)]
    targets=np.zeros((batchsize,))
    targets[batchsize//2:] = 1
    for i in range(0,batchsize):
        alph = alphabet[i]
        idx1 = np.random.randint(0,n_examples)
        pairs[0][i,:,:,:] = x[alph,idx1].reshape(width,height,1)
        idx2 = np.random.randint(0,n_examples)
        if i>=batchsize/2:
            pairs[1][i,:,:,:] = x[alph,idx2].reshape(width,height,1)
        else:
            alph = (alph + random.randint(1,n_chars))%n_chars
            pairs[1][i,:,:,:] = x[alph , idx2].reshape(width,height,1)
        
    return pairs,targets

In [18]:
def generator(batchsize):
    while True:
            pairs, targets = generate_mini_batch(batchsize, x)
            yield (pairs, targets)

## Build the model

In [85]:
def weight_init(shape , dtype=None , name=None):
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)
def bias_init(shape, dtype=None , name=None):
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [86]:
def build_model(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    print(left_input)
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=weight_init, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=weight_init,
                     bias_initializer=bias_init, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=weight_init,
                     bias_initializer=bias_init, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=weight_init,
                     bias_initializer=bias_init, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=weight_init,bias_initializer=bias_init))
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid',bias_initializer=bias_init)(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    return siamese_net

In [87]:
model = build_model((105, 105, 1))

KerasTensor(type_spec=TensorSpec(shape=(None, 105, 105, 1), dtype=tf.float32, name='input_11'), name='input_11', description="Symbolic value 0 from symbolic call 0 of layer 'input_11'")


In [22]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 4096)         38947648    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 4096)         0           sequential[0][0]      

In [23]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

## Initialise hyperparameters

In [24]:
batchsize = 32
iterations = 20000
for i in range(1, iterations+1):
    (inputs,targets) =generate_mini_batch(batchsize,x)
    loss = model.train_on_batch(inputs, targets)
    if i%100 == 0:
        print("Loss: " , loss , "Iteration: ", i)
        

Loss:  2.400463104248047 Iteration:  100
Loss:  1.5920823812484741 Iteration:  200
Loss:  1.213270902633667 Iteration:  300
Loss:  1.039536952972412 Iteration:  400
Loss:  0.8766752481460571 Iteration:  500
Loss:  0.8876118659973145 Iteration:  600
Loss:  0.6415328979492188 Iteration:  700
Loss:  0.7101261019706726 Iteration:  800
Loss:  0.46068334579467773 Iteration:  900
Loss:  0.601530134677887 Iteration:  1000
Loss:  0.5049608945846558 Iteration:  1100
Loss:  0.6391229033470154 Iteration:  1200
Loss:  0.5988340377807617 Iteration:  1300
Loss:  0.5784866809844971 Iteration:  1400
Loss:  0.5102099180221558 Iteration:  1500
Loss:  0.603340744972229 Iteration:  1600
Loss:  0.4500719904899597 Iteration:  1700
Loss:  0.4727054238319397 Iteration:  1800
Loss:  0.5489341616630554 Iteration:  1900
Loss:  0.390188604593277 Iteration:  2000
Loss:  0.5089820027351379 Iteration:  2100
Loss:  0.36753201484680176 Iteration:  2200
Loss:  0.364459753036499 Iteration:  2300
Loss:  0.3049468696117401

In [25]:
model.summary

<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7f0bcc758748>>

## Test data import

In [26]:
!unzip -uq "/content/drive/My Drive/siamese/images_test.zip" -d "/content/drive/My Drive/image_test"

In [46]:
%pylab inline
x_test = []
y_test = []
cat_dict = {}
lang_dict = {}
n=0
for a in os.listdir('/content/drive/My Drive/image_test/images_test'):
    print("Loading alphabet" , a)
    lang_dict[a] = [n , None]
    a_path = os.path.join('/content/drive/My Drive/image_test/images_test',a)
    
    for c in os.listdir(a_path):
        cat_dict[n] = (a,c)
        c_images = []
        c_path = os.path.join(a_path,c)
        
        for c_20 in os.listdir(c_path):
            c_20_path = os.path.join(c_path , c_20)
            img = imread(c_20_path)
            c_images.append(img)
            y_test.append(n)
            
        x_test.append(np.stack(c_images))
        lang_dict[a][1] = n
        n = n + 1       

Populating the interactive namespace from numpy and matplotlib
Loading alphabet Glagolitic
Loading alphabet Gurmukhi
Loading alphabet Kannada
Loading alphabet Keble
Loading alphabet Malayalam
Loading alphabet Manipuri
Loading alphabet Mongolian
Loading alphabet Old_Church_Slavonic_(Cyrillic)
Loading alphabet Oriya
Loading alphabet Sylheti
Loading alphabet Syriac_(Serto)
Loading alphabet Tengwar
Loading alphabet Tibetan
Loading alphabet ULOG
Loading alphabet Angelic
Loading alphabet Atemayar_Qelisayer
Loading alphabet Atlantean
Loading alphabet Aurek-Besh
Loading alphabet Avesta
Loading alphabet Ge_ez


In [48]:
x_test = np.stack(x_test)
y_test = np.vstack(y_test)

In [49]:
y_test.shape

(13180, 1)

In [33]:
lang_dict

{'Angelic': [509, 528],
 'Atemayar_Qelisayer': [529, 554],
 'Atlantean': [555, 580],
 'Aurek-Besh': [581, 606],
 'Avesta': [607, 632],
 'Ge_ez': [633, 658],
 'Glagolitic': [0, 44],
 'Gurmukhi': [45, 89],
 'Kannada': [90, 130],
 'Keble': [131, 156],
 'Malayalam': [157, 203],
 'Manipuri': [204, 243],
 'Mongolian': [244, 273],
 'Old_Church_Slavonic_(Cyrillic)': [274, 318],
 'Oriya': [319, 364],
 'Sylheti': [365, 392],
 'Syriac_(Serto)': [393, 415],
 'Tengwar': [416, 440],
 'Tibetan': [441, 482],
 'ULOG': [483, 508]}

In [51]:
def make_oneshot_task(N, x_test, lang_dict,language=None):
    X = x_test
    categories = lang_dict
    n_classes, n_examples, w, h = np.shape(X)
    indices = np.random.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = np.random.choice(range(low,high),size=(N,),replace=False)
    else: # if no language specified just pick a bunch of random letters
        categories = np.random.choice(range(n_classes),size=(N,),replace=False)            
    
    true_category = categories[0]
    ex1, ex2 = np.random.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    c = list(zip(targets, test_image ,support_set))
    random.shuffle(c)
    targets, test_image, support_set = zip(*c)
    pairs = [test_image,support_set]
    return pairs, targets


In [55]:
def test_model(model , N , k , x_test ,lang_dict):
  correct_pairs = 0
  for i in range(0,k):
    pairs,targets = make_oneshot_task(N,x_test,lang_dict)
    prediction = model.predict(pairs)
    if np.argmax(prediction) == np.argmax(targets):
      correct_pairs = correct_pairs + 1
  percent_correct = (100.0 * correct_pairs / k)
  print("Accuracy% : " , percent_correct)
  return percent_correct

In [83]:
K.set_image_data_format('channels_last')


In [ ]:
ways = np.arange(1,20,2)
k=50
for N in ways:
  print(N)
  test_model(model,N,k,x_test,lang_dict)